# Amazon Nova

In [18]:
import boto3

session = boto3.Session(profile_name="gen-ai-ops", region_name="us-east-1")
bedrock_client = session.client(service_name="bedrock")
bedrock_runtime_client = session.client(service_name="bedrock-runtime")